# <center> Wavefront set extractor using tensorflow Berkeley </center>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
import dense.shearlab as shearlab
import matplotlib.pyplot as plt
%matplotlib inline
import time
import dense.batchgen as bg
import h5py

In [40]:
# Lets create several data

In [5]:
size = 512
path = './BSR/BSDS500/data/'
dataset = 'train'
nClasses = 180
size_patch = 21

In [6]:
rows = size
cols = size
nScales = 4
shearletSystem = shearlab.getshearletsystem2D(rows,cols,nScales);

In [46]:
array1, label1 = bg.gen_batch_berkeley(path, dataset, size, nClasses, shearletSystem, size_patch)

In [47]:
array2, label2 = bg.gen_batch_berkeley(path, dataset, size, nClasses, shearletSystem, size_patch)

In [48]:
array3, label3 = bg.gen_batch_berkeley(path, dataset, size, nClasses, shearletSystem, size_patch)

In [49]:
array4, label4 = bg.gen_batch_berkeley(path, dataset, size, nClasses, shearletSystem, size_patch)

In [50]:
array5, label5 = bg.gen_batch_berkeley(path, 'test', size, nClasses, shearletSystem, size_patch)

In [51]:
array6, label6 = bg.gen_batch_berkeley(path, 'test', size, nClasses, shearletSystem, size_patch)

In [52]:
array7, label7 = bg.gen_batch_berkeley(path, 'test', size, nClasses, shearletSystem, size_patch)

In [53]:
array8, label8 = bg.gen_batch_berkeley(path, 'test', size, nClasses, shearletSystem, size_patch)

Concatenate the arrays and labels

In [54]:
array = np.concatenate((array1,array2,array3,array4, array5, array6, array8))
label = np.concatenate((label1,label2,label3,label4, label5, label6, label8))

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [7]:
import h5py

**Saving the dataset**

In [56]:
# create an hdf5 file
with h5py.File("array512berkeley.h5") as f:
    # create a dataset for your movie
    dst = f.create_dataset("array", shape=(array.shape[0], array.shape[1], 
                                           array.shape[2],array.shape[3]),
                           dtype=np.float64)
    for frame in range(array.shape[0]):
        dst[frame] = array[frame,:,:,:]

In [57]:
# create an hdf5 file
with h5py.File("label512berkeley.h5") as f:
    # create a dataset for your movie
    dst_label = f.create_dataset("label", shape=(label.shape[0], label.shape[1]),
                           dtype=np.float64)
    for frame in range(label.shape[0]):
        dst_label[frame] = label[frame,:]

**Loading dataset**

In [58]:
#Reading the data
with h5py.File("array512berkeley.h5", 'r') as h5:
    array = h5["array"][:]

In [59]:
#Reading the data
with h5py.File("label512berkeley.h5", 'r') as h5:
    label = h5["label"][:]

Do some random permutations

In [60]:
permutations = np.random.permutation(array.shape[0])

In [61]:
array = array[permutations,:,:,:]
label = label[permutations,:]

In [62]:
array.shape

(78436, 20, 20, 49)

In [63]:
label.shape

(78436, 181)

### Classifying the second angle


Lets create several data

In [64]:
def extract_angle(label,angle):
    return label[angle] == 1.0

In [65]:
def generate_training_test(array, label, angle):
    # Extract the position of the elements with that angle
    labels_angle = [extract_angle(labeli,angle) for labeli in label]
    # Generating the labels and arrays
    label_angle = np.ones(sum(labels_angle))
    array_angle = array[labels_angle]
    label_noangle = np.zeros(sum(labels_angle))
    array_noangle = array[~np.array(labels_angle)]
    # Subsampling the arrays without that angle
    array_noangle = array_noangle[0:sum(labels_angle)]
    # Creating the test, training and validation
    X = np.concatenate((array_angle,array_noangle))
    y = np.concatenate((label_angle,label_noangle))
    # Permutations
    permutations = np.random.permutation(X.shape[0])
    X = X[permutations]
    y = y[permutations]
    # Train test
    X_train = X[0:int(X.shape[0]*0.8-1)]
    y_train = y[0:int(y.shape[0]*0.8-1)]
    # Test set
    X_test = X[int(X.shape[0]*0.8):(X.shape[0]-1)]
    y_test = y[int(y.shape[0]*0.8):(y.shape[0]-1)]
    # Now validation
    X_valid = X_train[int(X_train.shape[0]*0.8):(X_train.shape[0]-1)]
    y_valid = y_train[int(y_train.shape[0]*0.8):(y_train.shape[0]-1)]
    X_train = X_train[0:int(X_train.shape[0]*0.8-1)]
    y_train = y_train[0:int(y_train.shape[0]*0.8-1)]
    
    return X_train, X_test, X_valid, y_train, y_test, y_valid

In [66]:
label.shape

(78436, 181)

In [67]:
# Classification of second angle
angle = 1

In [68]:
X_train, X_test, X_valid, y_train, y_test, y_valid = generate_training_test(array, label, angle)

In [69]:
X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape

((37383, 20, 20, 49),
 (11683, 20, 20, 49),
 (9346, 20, 20, 49),
 (37383,),
 (11683,),
 (9346,))

### Save data sets

In [70]:
# create an hdf5 file
with h5py.File('angle2berkeley.h5') as f:
    # create a dataset for your movie
    X_trainf = f.create_dataset("X_train", shape=(X_train.shape[0], X_train.shape[1], 
                                           X_train.shape[2],X_train.shape[3]),
                           dtype=np.float64)
    for frame in range(X_train.shape[0]):
        X_trainf[frame] = X_train[frame,:,:,:]
        
    X_validf = f.create_dataset("X_valid", shape=(X_valid.shape[0], X_valid.shape[1], 
                                           X_valid.shape[2],X_valid.shape[3]),
                           dtype=np.float64)
    for frame in range(X_valid.shape[0]):
        X_validf[frame] = X_valid[frame,:,:,:]
    
    X_testf = f.create_dataset("X_test", shape=(X_test.shape[0], X_test.shape[1], 
                                                 X_test.shape[2],X_test.shape[3]),
                           dtype=np.float64)
    for frame in range(X_test.shape[0]):
        X_testf[frame] = X_test[frame,:,:,:]
    
    y_trainf = f.create_dataset("y_train", shape=(y_train.shape[0],),
                           dtype=np.float64)
    for frame in range(y_train.shape[0]):
        y_trainf[frame] = y_train[frame]
    
    y_validf = f.create_dataset("y_valid", shape=(y_valid.shape[0],),
                           dtype=np.float64)
    for frame in range(y_valid.shape[0]):
        y_validf[frame] = y_valid[frame]

    y_testf = f.create_dataset("y_test", shape=(y_test.shape[0],),
                           dtype=np.float64)
    for frame in range(y_test.shape[0]):
        y_testf[frame] = y_test[frame]

### Load dataset

In [8]:
#Reading the data
with h5py.File("angle2berkeley.h5", 'r') as h5:
    X_train = h5["X_train"][:]
    X_test = h5["X_test"][:]
    X_valid = h5["X_valid"][:]
    y_train = h5["y_train"][:]
    y_test = h5["y_test"][:]
    y_valid = h5["y_valid"][:]

In [9]:
X_test = X_test.astype('float32')
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
y_test = y_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')

In [10]:
 X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape

((37383, 20, 20, 49),
 (11683, 20, 20, 49),
 (9346, 20, 20, 49),
 (37383,),
 (11683,),
 (9346,))

## Training

Lets create an architecture

In [11]:
### Training the model

In [12]:
from __future__ import division, print_function, absolute_import

In [13]:
from adler.tensorflow import prelu, cosine_decay

In [14]:
import os
import adler
adler.util.gpu.setup_one_gpu()
import warnings
warnings.filterwarnings('ignore')

Picking GPU 1


In [15]:
import tensorflow as tf
import numpy as np

In [19]:
from tensorflow.keras.utils import to_categorical

In [20]:
np.random.seed(0)
#name = os.path.splitext(os.path.basename(__file__))[0]
name = os.path.splitext(os.getcwd())[0]

**To categorical**

In [21]:
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)
y_valid = to_categorical(y_valid, num_classes = 2)

In [22]:
width = X_train.shape[1]
height = X_train.shape[2]
channels = X_train.shape[3]
nLabel = 2

In [23]:
## Weight Initialization
# Create lots of weights and biases & Initialize with a small positive number as we will use ReLU
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') 

## Pooling: max pooling over 2x2 blocks
def max_pool_2x2(x): 
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

**Architecture**: 4 convolutional layers and 2 fully connected

**Model**

In [24]:
conv_layers = 4
fully_connected = 1

In [25]:
graph = tf.Graph()
with graph.as_default():
    # Placeholders
    x = tf.placeholder(tf.float32, shape=(None, width, height,channels))
    y_ = tf.placeholder(tf.float32, shape=(None, nLabel))
    
    ## First Convolutional Layer
    W_conv1 = weight_variable([3, 3, 49, 49*4])
    b_conv1 = bias_variable([49*4])
    #Convolution
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    print('h_conv1',h_conv1.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean1, batch_var1 = tf.nn.moments(h_conv1,[0])
    h_conv1hat = (h_conv1-batch_mean1) / tf.sqrt(batch_var1 + 1e-3)
    # Pooling
    #h_pool1 = max_pool_2x2(h_conv1hat) 
    #print('h_pool1',h_pool1.shape)
    # No_pooling
    h_pool1 = h_conv1hat
    print('h_pool1',h_pool1.shape)
    
    ## Second Convolutional Layer
    W_conv2 = weight_variable([3, 3, 49*4, 49*4*4])
    b_conv2 = bias_variable([49*4*4])
    #Convolution
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    print('h_conv2',h_conv2.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean2, batch_var2 = tf.nn.moments(h_conv2,[0])
    h_conv2hat = (h_conv2-batch_mean2) / tf.sqrt(batch_var2 + 1e-3)
    
    # Pooling
    h_pool2 = max_pool_2x2(h_conv2hat) 
    print('h_pool2',h_pool2.shape)
    
    ## Third Convolutional Layer
    W_conv3 = weight_variable([3, 3, 49*4*4, 49*4*4*2])
    b_conv3 = bias_variable([49*4*4*2])
    #Convolution
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    print('h_conv3',h_conv3.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean3, batch_var3 = tf.nn.moments(h_conv3,[0])
    h_conv3hat = (h_conv3-batch_mean3) / tf.sqrt(batch_var3 + 1e-3)
    
    # Pooling
    h_pool3 = max_pool_2x2(h_conv3hat) 
    print('h_pool3',h_pool3.shape)

    
    ## Third Convolutional Layer
    W_conv4 = weight_variable([3, 3, 49*4*4*2, 49*4*4*2*2])
    b_conv4 = bias_variable([49*4*4*2*2])
    #Convolution
    h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
    print('h_conv4',h_conv4.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean4, batch_var4 = tf.nn.moments(h_conv4,[0])
    h_conv4hat = (h_conv4-batch_mean4) / tf.sqrt(batch_var4 + 1e-3)
    
    # Pooling
    h_pool4 = max_pool_2x2(h_conv4hat) 
    print('h_pool4',h_pool4.shape)

    ## Densely Connected Layer 

    # new shapes of pooled vectors
    _, width_pooled, height_pooled, channels_pooled = h_pool4.shape

    # fully-connected layer with 1024 neurons to process on the entire image
    W_fc1 = weight_variable([int(width_pooled*height_pooled*channels_pooled), 1024])  
    b_fc1 = bias_variable([1024])
    
    # Flat the output of the convolutional labels
    h_pool4_flat = tf.reshape(h_pool4, [-1, int(width_pooled*height_pooled*channels_pooled)])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)
    
    print('h_pool4_flat',h_pool4_flat.shape)

    ## Dropout (to reduce overfitting; useful when training very large neural network)
    # We will turn on dropout during training & turn off during testing
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    ## Readout Layer

    W_fc2 = weight_variable([1024, nLabel]) # [1024, 10]
    b_fc2 = bias_variable([nLabel]) # [10]
    
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # set up for optimization (optimizer:ADAM)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)  # 1e-4
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

h_conv1 (?, 20, 20, 196)
h_pool1 (?, 20, 20, 196)
h_conv2 (?, 20, 20, 784)
h_pool2 (?, 10, 10, 784)
h_conv3 (?, 10, 10, 1568)
h_pool3 (?, 5, 5, 1568)
h_conv4 (?, 5, 5, 3136)
h_pool4 (?, 3, 3, 3136)
h_pool4_flat (?, 28224)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Train the model


In [28]:
batch_size_train = 86
batch_size_valid = 1000
batch_size_test = 1000
num_steps = 40000

In [29]:
test_accuracy_minibatches = []
valid_accuracy_minibatches = []
best_validation_accuracy = 0.0
best_validation_accuracy_step = 0

In [31]:
with tf.Session(graph=graph) as session: 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    # Saver setup
    saver = tf.train.Saver()
    save_dir = 'checkpoints_berkeley/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, 'model_angle2')

    
    # Train the model
    for step in range(num_steps):
        # Logging every 100th iteration in the training process.
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size_train) % (y_train.shape[0] - batch_size_train)
        
        ## Lets do validation
        offset_valid = (step * batch_size_valid) % (y_valid.shape[0] - batch_size_valid)

        # Generate a minibatch.
        batch_data = X_train[offset:(offset + batch_size_train), :]
        batch_labels = y_train[offset:(offset + batch_size_train), :]
        if step%5 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch_data, y_: batch_labels, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(step, train_accuracy))
            
        if step%100 == 0:
            ## Some test
            # Generate a test minibatch.
            batch_data_valid = X_valid[offset_valid:(offset_valid + batch_size_valid), :]
            batch_labels_valid = y_valid[offset_valid:(offset_valid + batch_size_valid), :]
            valid_accuracy = accuracy.eval(feed_dict={x:batch_data_valid, y_: batch_labels_valid, keep_prob: 1.0})
            valid_accuracy_minibatches.append(valid_accuracy)
            print("\033[1m step %d, validation accuracy \033[0;0m %g"%(step, valid_accuracy))
            
            # Save when there is an improvement 
            if valid_accuracy > best_validation_accuracy:
                print("\033[1m The validation accuracy has improved from the last checkpoint \033[0;0m")
                best_validation_accuracy = valid_accuracy
                best_validation_accuracy_step = step
                saver.save(sess = session, save_path=save_path)
            else:
                print("\033[1m The validation accuracy DID NOT improved from the last checkpoint \033[0;0m")

            
        train_step.run(feed_dict={x: batch_data, y_: batch_labels, keep_prob: 0.5})
        
        
    # Evaulate our accuracy on the test data
    # Test the model
    for step in range(num_steps):
        offset_test = (step * batch_size_test) % (y_test.shape[0] - batch_size_test)

        # Generate a minibatch.
        batch_data_test = X_test[offset_test:(offset_test + batch_size_test), :]
        batch_labels_test = y_test[offset_test:(offset_test + batch_size_test), :]
        test_accuracy = accuracy.eval(feed_dict={x:batch_data_test, y_: batch_labels_test, keep_prob: 1.0})
        test_accuracy_minibatches.append(test_accuracy)
        
        if step%5 == 0:
            print("step %d, test accuracy %g"%(step, test_accuracy))

Initialized
step 0, training accuracy 0.5
 step 0, validation accuracy  0.507
 The validation accuracy has improved from the last checkpoint 
step 5, training accuracy 0.5
step 10, training accuracy 0.616279
step 15, training accuracy 0.651163
step 20, training accuracy 0.604651
step 25, training accuracy 0.406977
step 30, training accuracy 0.581395
step 35, training accuracy 0.581395
step 40, training accuracy 0.546512
step 45, training accuracy 0.465116
step 50, training accuracy 0.569767
step 55, training accuracy 0.639535
step 60, training accuracy 0.476744
step 65, training accuracy 0.616279
step 70, training accuracy 0.523256
step 75, training accuracy 0.488372
step 80, training accuracy 0.604651
step 85, training accuracy 0.639535
step 90, training accuracy 0.569767
step 95, training accuracy 0.569767
step 100, training accuracy 0.5
 step 100, validation accuracy  0.596
 The validation accuracy has improved from the last checkpoint 
step 105, training accuracy 0.546512
step 110,

step 950, training accuracy 0.709302
step 955, training accuracy 0.697674
step 960, training accuracy 0.72093
step 965, training accuracy 0.744186
step 970, training accuracy 0.72093
step 975, training accuracy 0.72093
step 980, training accuracy 0.72093
step 985, training accuracy 0.674419
step 990, training accuracy 0.72093
step 995, training accuracy 0.744186
step 1000, training accuracy 0.755814
 step 1000, validation accuracy  0.731
 The validation accuracy has improved from the last checkpoint 
step 1005, training accuracy 0.627907
step 1010, training accuracy 0.662791
step 1015, training accuracy 0.651163
step 1020, training accuracy 0.674419
step 1025, training accuracy 0.697674
step 1030, training accuracy 0.77907
step 1035, training accuracy 0.790698
step 1040, training accuracy 0.732558
step 1045, training accuracy 0.697674
step 1050, training accuracy 0.732558
step 1055, training accuracy 0.662791
step 1060, training accuracy 0.72093
step 1065, training accuracy 0.72093
ste

step 1885, training accuracy 0.674419
step 1890, training accuracy 0.697674
step 1895, training accuracy 0.744186
step 1900, training accuracy 0.755814
 step 1900, validation accuracy  0.742
 The validation accuracy DID NOT improved from the last checkpoint 
step 1905, training accuracy 0.755814
step 1910, training accuracy 0.709302
step 1915, training accuracy 0.651163
step 1920, training accuracy 0.709302
step 1925, training accuracy 0.813953
step 1930, training accuracy 0.732558
step 1935, training accuracy 0.732558
step 1940, training accuracy 0.755814
step 1945, training accuracy 0.697674
step 1950, training accuracy 0.813953
step 1955, training accuracy 0.744186
step 1960, training accuracy 0.767442
step 1965, training accuracy 0.732558
step 1970, training accuracy 0.755814
step 1975, training accuracy 0.744186
step 1980, training accuracy 0.616279
step 1985, training accuracy 0.616279
step 1990, training accuracy 0.744186
step 1995, training accuracy 0.77907
step 2000, training 

step 2805, training accuracy 0.72093
step 2810, training accuracy 0.813953
step 2815, training accuracy 0.744186
step 2820, training accuracy 0.744186
step 2825, training accuracy 0.732558
step 2830, training accuracy 0.732558
step 2835, training accuracy 0.744186
step 2840, training accuracy 0.77907
step 2845, training accuracy 0.709302
step 2850, training accuracy 0.72093
step 2855, training accuracy 0.697674
step 2860, training accuracy 0.732558
step 2865, training accuracy 0.686047
step 2870, training accuracy 0.744186
step 2875, training accuracy 0.72093
step 2880, training accuracy 0.709302
step 2885, training accuracy 0.755814
step 2890, training accuracy 0.604651
step 2895, training accuracy 0.72093
step 2900, training accuracy 0.616279
 step 2900, validation accuracy  0.724
 The validation accuracy DID NOT improved from the last checkpoint 
step 2905, training accuracy 0.755814
step 2910, training accuracy 0.72093
step 2915, training accuracy 0.755814
step 2920, training accur

step 3740, training accuracy 0.790698
step 3745, training accuracy 0.72093
step 3750, training accuracy 0.755814
step 3755, training accuracy 0.674419
step 3760, training accuracy 0.77907
step 3765, training accuracy 0.744186
step 3770, training accuracy 0.697674
step 3775, training accuracy 0.697674
step 3780, training accuracy 0.709302
step 3785, training accuracy 0.697674
step 3790, training accuracy 0.77907
step 3795, training accuracy 0.662791
step 3800, training accuracy 0.744186
 step 3800, validation accuracy  0.741
 The validation accuracy DID NOT improved from the last checkpoint 
step 3805, training accuracy 0.755814
step 3810, training accuracy 0.639535
step 3815, training accuracy 0.744186
step 3820, training accuracy 0.825581
step 3825, training accuracy 0.709302
step 3830, training accuracy 0.686047
step 3835, training accuracy 0.662791
step 3840, training accuracy 0.697674
step 3845, training accuracy 0.732558
step 3850, training accuracy 0.686047
step 3855, training ac

...

step 39440, training accuracy 0.94186
step 39445, training accuracy 0.988372
step 39450, training accuracy 0.988372
step 39455, training accuracy 0.976744
step 39460, training accuracy 0.988372
step 39465, training accuracy 0.988372
step 39470, training accuracy 1
step 39475, training accuracy 1
step 39480, training accuracy 0.988372
step 39485, training accuracy 0.988372
step 39490, training accuracy 0.976744
step 39495, training accuracy 0.988372
step 39500, training accuracy 0.988372
 step 39500, validation accuracy  0.974
 The validation accuracy DID NOT improved from the last checkpoint 
step 39505, training accuracy 0.930233
step 39510, training accuracy 1
step 39515, training accuracy 0.965116
step 39520, training accuracy 0.988372
step 39525, training accuracy 1
step 39530, training accuracy 0.953488
step 39535, training accuracy 0.988372
step 39540, training accuracy 0.953488
step 39545, training accuracy 0.965116
step 39550, training accuracy 0.965116
step 39555, training acc

step 560, test accuracy 0.978
step 565, test accuracy 0.973
step 570, test accuracy 0.974
step 575, test accuracy 0.96
step 580, test accuracy 0.973
step 585, test accuracy 0.981
step 590, test accuracy 0.976
step 595, test accuracy 0.974
step 600, test accuracy 0.969
step 605, test accuracy 0.974
step 610, test accuracy 0.978
step 615, test accuracy 0.975
step 620, test accuracy 0.978
step 625, test accuracy 0.975
step 630, test accuracy 0.97
step 635, test accuracy 0.978
step 640, test accuracy 0.975
step 645, test accuracy 0.971
step 650, test accuracy 0.982
step 655, test accuracy 0.976
step 660, test accuracy 0.967
step 665, test accuracy 0.968
step 670, test accuracy 0.979
step 675, test accuracy 0.978
step 680, test accuracy 0.973
step 685, test accuracy 0.976
step 690, test accuracy 0.974
step 695, test accuracy 0.982
step 700, test accuracy 0.979
step 705, test accuracy 0.977
step 710, test accuracy 0.973
step 715, test accuracy 0.978
step 720, test accuracy 0.963
step 725, te

step 1900, test accuracy 0.975
step 1905, test accuracy 0.977
step 1910, test accuracy 0.974
step 1915, test accuracy 0.978
step 1920, test accuracy 0.981
step 1925, test accuracy 0.975
step 1930, test accuracy 0.975
step 1935, test accuracy 0.977
step 1940, test accuracy 0.959
step 1945, test accuracy 0.973
step 1950, test accuracy 0.982
step 1955, test accuracy 0.971
step 1960, test accuracy 0.979
step 1965, test accuracy 0.975
step 1970, test accuracy 0.972
step 1975, test accuracy 0.965
step 1980, test accuracy 0.978
step 1985, test accuracy 0.974
step 1990, test accuracy 0.973
step 1995, test accuracy 0.972
step 2000, test accuracy 0.975
step 2005, test accuracy 0.977
step 2010, test accuracy 0.98
step 2015, test accuracy 0.975
step 2020, test accuracy 0.975
step 2025, test accuracy 0.978
step 2030, test accuracy 0.964
step 2035, test accuracy 0.975
step 2040, test accuracy 0.975
step 2045, test accuracy 0.978
step 2050, test accuracy 0.973
step 2055, test accuracy 0.968
step 2060

step 3225, test accuracy 0.98
step 3230, test accuracy 0.976
step 3235, test accuracy 0.98
step 3240, test accuracy 0.975
step 3245, test accuracy 0.976
step 3250, test accuracy 0.966
step 3255, test accuracy 0.978
step 3260, test accuracy 0.977
step 3265, test accuracy 0.971
step 3270, test accuracy 0.979
step 3275, test accuracy 0.975
step 3280, test accuracy 0.973
step 3285, test accuracy 0.972
step 3290, test accuracy 0.979
step 3295, test accuracy 0.973
step 3300, test accuracy 0.977
step 3305, test accuracy 0.974
step 3310, test accuracy 0.972
step 3315, test accuracy 0.977
step 3320, test accuracy 0.978
step 3325, test accuracy 0.971
step 3330, test accuracy 0.973
step 3335, test accuracy 0.972
step 3340, test accuracy 0.962
step 3345, test accuracy 0.973
step 3350, test accuracy 0.977
step 3355, test accuracy 0.977
step 3360, test accuracy 0.978
step 3365, test accuracy 0.971
step 3370, test accuracy 0.973
step 3375, test accuracy 0.977
step 3380, test accuracy 0.974
step 3385,

KeyboardInterrupt: 

### Summary of the method

In [3]:
print("The average test accuracy is "+str(sum(test_accuracy_minibatches)/len(test_accuracy_minibatches)*100))

The average test accuracy is 95.35699855112
